# Single-Cell Report: Scrublet - Doublet Detection

In [ ]:
# Import packages
import numpy as np
import scanpy as sc
import scrublet as scr
import matplotlib.pyplot as plt
import gzip
import pickle

In [ ]:
sc.set_figure_params(dpi=150, fontsize=10, dpi_save=600)

In [ ]:
f = gzip.open(SCRUBLET_OBJECT_FILE,'rb')
scrub = pickle.load(f)
f.close()

In [ ]:
adata_sl = sc.read_h5ad(H5AD_WITH_SCRUBLET_INFO)

In [ ]:
adata_dr = sc.read_h5ad(H5AD_WITH_DIM_RED)

In [ ]:
print(adata_dr.obs.isna().sum())

## Merge data

In [ ]:
# Keep only Scrublet columns from the Scrublet AnnData this will avoid #301
scrublet_columns_to_keep = list(filter(lambda column: column.startswith("scrublet__"), adata_sl.obs.columns))
adata_sl_obs = adata_sl.obs.loc[:, scrublet_columns_to_keep]
import numpy as np
if any(np.in1d(adata_sl_obs.columns, adata_dr.obs.columns)):
    raise Exception(f"VSN ERROR: Overlapping columns detected. The following columns should be removed in the source file: {', '.join(scrublet_columns_to_keep)}")

In [ ]:
# Merge AnnData DR obs with AnnData Sl (Scrublet) obs
adata_dr.obs = adata_dr.obs.join(
    other=adata_sl_obs
)

In [ ]:
adata_dr.obs.head()

In [ ]:
# Deal with missing values

In [ ]:
values = {
    'scrublet__doublet_scores': -0.01,
    'scrublet__predicted_doublets': False,
    'scrublet__predicted_doublets_based_on_10x_chromium_spec': False,
}
adata_dr.obs = adata_dr.obs.fillna(
    value=values
)

## Plot doublet score histograms for observed transcriptomes and simulated doublets

In [ ]:
# More info: https://github.com/swolock/scrublet/blob/master/examples/scrublet_basics.ipynb

In [ ]:
if scrub.predicted_doublets_ is None:
    print("No doublets predicted using default method.")
else:
    scrub.plot_histogram()

## Plot doublet predictions on 2-D embedding

### Doublet Scores

In [ ]:
a = 0.6 # alpha setting
fig, (axs) = plt.subplots(1,2, figsize=(10,5), dpi=150 )
ax1 = sc.pl.tsne(adata_dr, color='scrublet__doublet_scores',size=10,color_map='plasma_r',alpha=a, ax=axs[0], show=False, wspace=0.5)
ax1.set_title(f"Scrublet - Doublet Scores")
ax2 = sc.pl.umap(adata_dr, color='scrublet__doublet_scores',size=10,color_map='plasma_r',alpha=a, ax=axs[1], show=False, wspace=0.5)
ax2.set_title(f"Scrublet - Doublet Scores")
plt.tight_layout()

### Predicted Doublets (default)

In [ ]:
if scrub.predicted_doublets_ is None:
    print("No doublets predicted using default method.")
else:
    default_num_predicted_doublets = sum(adata_dr.obs['scrublet__predicted_doublets'])
    print(f"Number of predicted doublets (default): {default_num_predicted_doublets}")

In [ ]:
if scrub.predicted_doublets_ is None:
    print("No doublets predicted using default method.")
else:
    adata_dr.obs['scrublet__predicted_doublets_cat'] = adata_dr.obs['scrublet__predicted_doublets'].astype(str).astype('category')
    a = 0.6 # alpha setting
    fig, (axs) = plt.subplots(1,2, figsize=(10,5), dpi=150 )
    ax1 = sc.pl.tsne(adata_dr, color='scrublet__predicted_doublets_cat',size=10, color_map='plasma_r', alpha=a, ax=axs[0], show=False, wspace=0.5)
    ax1.set_title(f"Scrublet - Predicted Doublet")
    ax2 = sc.pl.umap(adata_dr, color='scrublet__predicted_doublets_cat',size=10, color_map='plasma_r', alpha=a, ax=axs[1], show=False, wspace=0.5)
    ax2.set_title(f"Scrublet - Predicted Doublet")
    plt.tight_layout()

### Predicted Doublet (w/ 10xGenomics Chemistry Specification)

In [ ]:
default_num_predicted_doublets_with_10x_chemistry_spec = sum(adata_dr.obs['scrublet__predicted_doublets_based_on_10x_chromium_spec'])
print(f"Number of predicted doublets (using 10xGenomics chemistry specification): {default_num_predicted_doublets_with_10x_chemistry_spec}")

In [ ]:
adata_dr.obs['scrublet__predicted_doublets_based_on_10x_chromium_spec_cat'] = adata_dr.obs['scrublet__predicted_doublets_based_on_10x_chromium_spec'].astype(str).astype('category')
a = 0.6 # alpha setting
fig, (axs) = plt.subplots(1,2, figsize=(10,5), dpi=150 )
ax1 = sc.pl.tsne(adata_dr, color='scrublet__predicted_doublets_based_on_10x_chromium_spec_cat',size=10, color_map='plasma_r',alpha=a, ax=axs[0], show=False, wspace=0.5)
ax1.set_title(f"Scrublet - Predicted Doublet")
ax2 = sc.pl.umap(adata_dr, color='scrublet__predicted_doublets_based_on_10x_chromium_spec_cat',size=10, color_map='plasma_r',alpha=a, ax=axs[1], show=False, wspace=0.5)
ax2.set_title(f"Scrublet - Predicted Doublet")
plt.tight_layout()